In [11]:
"""
COMPLETE STEP 2: Better Features Pipeline
Full pipeline with advanced feature engineering

Expected: 0.62 → 0.59-0.60 RMSE
Time: ~4-5 hours (training)

WHAT THIS DOES:
1. Load and prepare data
2. Add ADVANCED FEATURES (30-40 new features)
   - Distance rate features
   - Target alignment features
   - Multi-window rolling features
   - Extended lag features
   - Velocity change features
   - Field position features
   - Role-specific features
   - Time-based features
3. Train NN models with enhanced features
4. Create submission

"""

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from datetime import datetime
import warnings
import os
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from torch.utils.data import TensorDataset, DataLoader

warnings.filterwarnings('ignore')

# ============================================================================
# CONFIG
# ============================================================================

class Config:
    DATA_DIR = Path("./data")
    OUTPUT_DIR = Path("./outputs")
    OUTPUT_DIR.mkdir(exist_ok=True)
    
    SEED = 42
    N_FOLDS = 5
    BATCH_SIZE = 128
    EPOCHS = 200
    PATIENCE = 20
    LEARNING_RATE = 5e-4
    
    WINDOW_SIZE = 20
    HIDDEN_DIM = 64
    MAX_FUTURE_HORIZON = 94
    
    FIELD_X_MIN, FIELD_X_MAX = 0.0, 120.0
    FIELD_Y_MIN, FIELD_Y_MAX = 0.0, 53.3
    
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_seed(seed=42):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(Config.SEED)


In [ ]:

# ============================================================================
# ENHANCED FEATURE ENGINEERING
# ============================================================================

def height_to_feet(height_str):
    try:
        ft, inches = map(int, str(height_str).split('-'))
        return ft + inches/12
    except:
        return 6.0

def add_advanced_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    STEP 2: Add 30-40 advanced features
    These are proven to improve performance
    """
    print("Adding advanced features...")
    df = df.copy()
    df = df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    gcols = ['game_id', 'play_id', 'nfl_id']
    
    # ==========================================
    # GROUP 1: Distance Rate Features (3)
    # ==========================================
    if 'distance_to_ball' in df.columns:
        df['distance_to_ball_change'] = df.groupby(gcols)['distance_to_ball'].diff().fillna(0)
        df['distance_to_ball_accel'] = df.groupby(gcols)['distance_to_ball_change'].diff().fillna(0)
        df['time_to_intercept'] = (df['distance_to_ball'] / 
                                    (np.abs(df['distance_to_ball_change']) + 0.1)).clip(0, 10)
    
    # ==========================================
    # GROUP 2: Target Alignment Features (3)
    # ==========================================
    if 'ball_direction_x' in df.columns:
        df['velocity_alignment'] = (
            df['velocity_x'] * df['ball_direction_x'] +
            df['velocity_y'] * df['ball_direction_y']
        )
        df['velocity_perpendicular'] = (
            df['velocity_x'] * (-df['ball_direction_y']) +
            df['velocity_y'] * df['ball_direction_x']
        )
        if 'acceleration_x' in df.columns:
            df['accel_alignment'] = (
                df['acceleration_x'] * df['ball_direction_x'] +
                df['acceleration_y'] * df['ball_direction_y']
            )
    
    # ==========================================
    # GROUP 3: Multi-Window Rolling (24)
    # ==========================================
    for window in [3, 5, 10]:
        for col in ['velocity_x', 'velocity_y', 's', 'a']:
            if col in df.columns:
                df[f'{col}_roll{window}'] = df.groupby(gcols)[col].transform(
                    lambda x: x.rolling(window, min_periods=1).mean()
                )
                df[f'{col}_std{window}'] = df.groupby(gcols)[col].transform(
                    lambda x: x.rolling(window, min_periods=1).std()
                ).fillna(0)
    
    # ==========================================
    # GROUP 4: Extended Lag Features (8)
    # ==========================================
    for lag in [4, 5]:
        for col in ['x', 'y', 'velocity_x', 'velocity_y']:
            if col in df.columns:
                df[f'{col}_lag{lag}'] = df.groupby(gcols)[col].shift(lag).fillna(0)
    
    # ==========================================
    # GROUP 5: Velocity Change Features (4)
    # ==========================================
    if 'velocity_x' in df.columns:
        df['velocity_x_change'] = df.groupby(gcols)['velocity_x'].diff().fillna(0)
        df['velocity_y_change'] = df.groupby(gcols)['velocity_y'].diff().fillna(0)
        df['speed_change'] = df.groupby(gcols)['s'].diff().fillna(0)
        df['direction_change'] = df.groupby(gcols)['dir'].diff().fillna(0)
        # df['direction_change'] = df['direction_change'].apply(
        #     lambda x: x if abs(x) < 180 else x - 360 * np.sign(x)
        # )
        dir_diff = df.groupby(gcols)['dir'].diff().fillna(0)
        df['direction_change'] = (((dir_diff + 180) % 360) - 180)
    
    # ==========================================
    # GROUP 6: Field Position Features (4)
    # ==========================================
    df['dist_from_left'] = df['y']
    df['dist_from_right'] = 53.3 - df['y']
    df['dist_from_sideline'] = np.minimum(df['dist_from_left'], df['dist_from_right'])
    df['dist_from_endzone'] = np.minimum(df['x'], 120 - df['x'])
    
    # ==========================================
    # GROUP 7: Role-Specific Features (3)
    # ==========================================
    if 'is_receiver' in df.columns and 'velocity_alignment' in df.columns:
        df['receiver_optimality'] = df['is_receiver'] * df['velocity_alignment']
        df['receiver_deviation'] = df['is_receiver'] * np.abs(df.get('velocity_perpendicular', 0))
    if 'is_coverage' in df.columns and 'closing_speed' in df.columns:
        df['defender_closing_speed'] = df['is_coverage'] * df['closing_speed']
    
    # ==========================================
    # GROUP 8: Time Features (2)
    # ==========================================
    df['frames_elapsed'] = df.groupby(gcols).cumcount()
    df['normalized_time'] = df.groupby(gcols)['frames_elapsed'].transform(
        lambda x: x / (x.max() + 1)
    )
    
    print(f"Total features after enhancement: {len(df.columns)}")
    
    return df



In [ ]:
def prepare_sequences_with_advanced_features(input_df: pd.DataFrame, 
                                             output_df: pd.DataFrame = None, 
                                             test_template: pd.DataFrame = None,
                                             is_training: bool =True, 
                                             window_size: int = 8):
    """
    Prepare sequences with ALL advanced features

    Output:
      - If is_training:
          sequences: List of np.arrays (num_sequences, window_size, num_features)
          targets_dx: List of np.arrays (num_sequences, num_future_frames)
          targets_dy: List of np.arrays (num_sequences, num_future_frames)
          targets_frame_ids: List of np.arrays (num_sequences, num_future_frames)
          sequence_ids: List of sequence identifiers
      - If not is_training:
          sequences: List of np.arrays (num_sequences, window_size, num_features)
          sequence_ids: List of sequence identifiers - game_id, play_id, nfl_id, frame_id
    """
    print(f"\n{'='*80}")
    print(f"PREPARING SEQUENCES WITH ADVANCED FEATURES")
    print(f"{'='*80}")
    print(f"Window size: {window_size}")
    
    input_df = input_df.copy()
    
    # ==========================================
    # BASIC FEATURES
    # ==========================================
    print("Step 1/3: Adding basic features...")
    
    input_df['player_height_feet'] = input_df['player_height'].apply(height_to_feet)
    
    dir_rad = np.deg2rad(input_df['dir'].fillna(0))
    delta_t = 0.1
    input_df['velocity_x'] = (input_df['s'] + 0.5 * input_df['a'] * delta_t) * np.sin(dir_rad)
    input_df['velocity_y'] = (input_df['s'] + 0.5 * input_df['a'] * delta_t) * np.cos(dir_rad)
    input_df['acceleration_x'] = input_df['a'] * np.sin(dir_rad)
    input_df['acceleration_y'] = input_df['a'] * np.cos(dir_rad)
    
    # Roles
    input_df['is_offense'] = (input_df['player_side'] == 'Offense').astype(int)
    input_df['is_defense'] = (input_df['player_side'] == 'Defense').astype(int)
    input_df['is_receiver'] = (input_df['player_role'] == 'Targeted Receiver').astype(int)
    input_df['is_coverage'] = (input_df['player_role'] == 'Defensive Coverage').astype(int)
    input_df['is_passer'] = (input_df['player_role'] == 'Passer').astype(int)
    
    # Physics
    mass_kg = input_df['player_weight'].fillna(200.0) / 2.20462
    input_df['momentum_x'] = input_df['velocity_x'] * mass_kg
    input_df['momentum_y'] = input_df['velocity_y'] * mass_kg
    input_df['kinetic_energy'] = 0.5 * mass_kg * (input_df['s'] ** 2)
    
    # Ball features
    if 'ball_land_x' in input_df.columns:
        ball_dx = input_df['ball_land_x'] - input_df['x']
        ball_dy = input_df['ball_land_y'] - input_df['y']
        input_df['distance_to_ball'] = np.sqrt(ball_dx**2 + ball_dy**2)
        input_df['angle_to_ball'] = np.arctan2(ball_dy, ball_dx)
        input_df['ball_direction_x'] = ball_dx / (input_df['distance_to_ball'] + 1e-6)
        input_df['ball_direction_y'] = ball_dy / (input_df['distance_to_ball'] + 1e-6)
        input_df['closing_speed'] = (
            input_df['velocity_x'] * input_df['ball_direction_x'] +
            input_df['velocity_y'] * input_df['ball_direction_y']
        )
    
    # Sort for temporal
    input_df = input_df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
    gcols = ['game_id', 'play_id', 'nfl_id']
    
    # Original lag features (1-3)
    for lag in [1, 2, 3]:
        input_df[f'x_lag{lag}'] = input_df.groupby(gcols)['x'].shift(lag)
        input_df[f'y_lag{lag}'] = input_df.groupby(gcols)['y'].shift(lag)
        input_df[f'velocity_x_lag{lag}'] = input_df.groupby(gcols)['velocity_x'].shift(lag)
        input_df[f'velocity_y_lag{lag}'] = input_df.groupby(gcols)['velocity_y'].shift(lag)
    
    # EMA features
    input_df['velocity_x_ema'] = input_df.groupby(gcols)['velocity_x'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    input_df['velocity_y_ema'] = input_df.groupby(gcols)['velocity_y'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    input_df['speed_ema'] = input_df.groupby(gcols)['s'].transform(
        lambda x: x.ewm(alpha=0.3, adjust=False).mean()
    )
    
    # ==========================================
    # ADVANCED FEATURES (NEW!)
    # ==========================================
    print("Step 2/3: Adding advanced features...")
    input_df = add_advanced_features(input_df)
    
    # ==========================================
    # FEATURE LIST (ENHANCED)
    # ==========================================
    print("Step 3/3: Creating sequences...")
    
    feature_cols = [
        # Core (9)
        'x', 'y', 's', 'a', 'o', 'dir', 'frame_id', 'ball_land_x', 'ball_land_y',
        
        # Player (2)
        'player_height_feet', 'player_weight',
        
        # Motion (6)
        'velocity_x', 'velocity_y', 'acceleration_x', 'acceleration_y',
        'momentum_x', 'momentum_y', 'kinetic_energy',
        
        # Roles (5)
        'is_offense', 'is_defense', 'is_receiver', 'is_coverage', 'is_passer',
        
        # Ball (5)
        'distance_to_ball', 'angle_to_ball', 'ball_direction_x', 'ball_direction_y', 'closing_speed',
        
        # Original temporal (15)
        'x_lag1', 'y_lag1', 'velocity_x_lag1', 'velocity_y_lag1',
        'x_lag2', 'y_lag2', 'velocity_x_lag2', 'velocity_y_lag2',
        'x_lag3', 'y_lag3', 'velocity_x_lag3', 'velocity_y_lag3',
        'velocity_x_ema', 'velocity_y_ema', 'speed_ema',
        
        # NEW: Distance rate (3)
        'distance_to_ball_change', 'distance_to_ball_accel', 'time_to_intercept',
        
        # NEW: Target alignment (3)
        'velocity_alignment', 'velocity_perpendicular', 'accel_alignment',
        
        # NEW: Multi-window rolling (24)
        'velocity_x_roll3', 'velocity_x_std3', 'velocity_y_roll3', 'velocity_y_std3',
        's_roll3', 's_std3', 'a_roll3', 'a_std3',
        'velocity_x_roll5', 'velocity_x_std5', 'velocity_y_roll5', 'velocity_y_std5',
        's_roll5', 's_std5', 'a_roll5', 'a_std5',
        'velocity_x_roll10', 'velocity_x_std10', 'velocity_y_roll10', 'velocity_y_std10',
        's_roll10', 's_std10', 'a_roll10', 'a_std10',
        
        # NEW: Extended lags (8)
        'x_lag4', 'y_lag4', 'velocity_x_lag4', 'velocity_y_lag4',
        'x_lag5', 'y_lag5', 'velocity_x_lag5', 'velocity_y_lag5',
        
        # NEW: Velocity changes (4)
        'velocity_x_change', 'velocity_y_change', 'speed_change', 'direction_change',
        
        # NEW: Field position (4)
        'dist_from_sideline', 'dist_from_endzone',
        
        # NEW: Role-specific (3)
        'receiver_optimality', 'receiver_deviation', 'defender_closing_speed',
        
        # NEW: Time (2)
        'frames_elapsed', 'normalized_time',
    ]
    
    # Filter to existing
    feature_cols = [c for c in feature_cols if c in input_df.columns]
    print(f"Using {len(feature_cols)} features (was ~50, now ~90)")
    
    # ==========================================
    # CREATE SEQUENCES
    # ==========================================
    input_df.set_index(['game_id', 'play_id', 'nfl_id'], inplace=True)
    grouped = input_df.groupby(level=['game_id', 'play_id', 'nfl_id'])
    
    target_rows = output_df if is_training else test_template # This is "output" for training or test set
    target_groups = target_rows[['game_id', 'play_id', 'nfl_id']].drop_duplicates()
    
    sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids = [], [], [], [], []
    
    for _, row in tqdm(target_groups.iterrows(), total=len(target_groups), desc="Creating sequences"):
        key = (row['game_id'], row['play_id'], row['nfl_id'])
        
        try:
            group_df = grouped.get_group(key)
        except KeyError:
            continue
        
        input_window = group_df.tail(window_size)
        
        if len(input_window) < window_size:
            if is_training:
                continue
            pad_len = window_size - len(input_window)
            pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=input_window.columns)
            input_window = pd.concat([pad_df, input_window], ignore_index=True)
        
        input_window = input_window.fillna(group_df.mean(numeric_only=True))
        seq = input_window[feature_cols].values
        
        if np.isnan(seq).any():
            if is_training:
                continue
            seq = np.nan_to_num(seq, nan=0.0)
        
        sequences.append(seq)
        
        if is_training:
            out_grp = output_df[
                (output_df['game_id']==row['game_id']) &
                (output_df['play_id']==row['play_id']) &
                (output_df['nfl_id']==row['nfl_id'])
            ].sort_values('frame_id')
            
            last_x = input_window.iloc[-1]['x']
            last_y = input_window.iloc[-1]['y']
            
            dx = out_grp['x'].values - last_x
            dy = out_grp['y'].values - last_y
            
            targets_dx.append(dx)
            targets_dy.append(dy)
            targets_frame_ids.append(out_grp['frame_id'].values)
        
        sequence_ids.append({
            'game_id': key[0],
            'play_id': key[1],
            'nfl_id': key[2],
            'frame_id': input_window.iloc[-1]['frame_id']
        })
    
    print(f"Created {len(sequences)} sequences with {len(feature_cols)} features each")
    
    if is_training:
        return sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids
    return sequences, sequence_ids



In [12]:
# ============================================================================
# MODEL & LOSS (Same as before)
# ============================================================================

class TemporalHuber(nn.Module):
    def __init__(self, delta=0.5, time_decay=0.03):
        super().__init__()
        self.delta = delta
        self.time_decay = time_decay
    
    def forward(self, pred, target, mask):
        err = pred - target
        abs_err = torch.abs(err)
        huber = torch.where(abs_err <= self.delta, 0.5 * err * err, 
                           self.delta * (abs_err - 0.5 * self.delta))
        
        if self.time_decay > 0:
            L = pred.size(1)
            t = torch.arange(L, device=pred.device).float()
            weight = torch.exp(-self.time_decay * t).view(1, L)
            huber, mask = huber * weight, mask * weight
        
        return (huber * mask).sum() / (mask.sum() + 1e-8)

class SeqModel(nn.Module):
    def __init__(self, input_dim, horizon):
        super().__init__()
        self.gru = nn.GRU(input_dim, 128, num_layers=2, batch_first=True, dropout=0.1)
        self.pool_ln = nn.LayerNorm(128)
        self.pool_attn = nn.MultiheadAttention(128, num_heads=4, batch_first=True)
        self.pool_query = nn.Parameter(torch.randn(1, 1, 128))
        self.head = nn.Sequential(
            nn.Linear(128, 128), nn.GELU(), nn.Dropout(0.2), nn.Linear(128, horizon)
        )
    
    def forward(self, x):
        h, _ = self.gru(x)
        B = h.size(0)
        q = self.pool_query.expand(B, -1, -1)
        ctx, _ = self.pool_attn(q, self.pool_ln(h), self.pool_ln(h))
        out = self.head(ctx.squeeze(1))
        return torch.cumsum(out, dim=1)

# ============================================================================
# TRAINING
# ============================================================================

def prepare_targets(batch_axis, max_h):
    tensors, masks = [], []
    for arr in batch_axis:
        L = len(arr)
        padded = np.pad(arr, (0, max_h - L), constant_values=0).astype(np.float32)
        mask = np.zeros(max_h, dtype=np.float32)
        mask[:L] = 1.0
        tensors.append(torch.tensor(padded))
        masks.append(torch.tensor(mask))
    return torch.stack(tensors), torch.stack(masks)

def train_model(X_train, y_train, X_val, y_val, input_dim, horizon, config):
    device = config.DEVICE
    model = SeqModel(input_dim, horizon).to(device)
    
    criterion = TemporalHuber(delta=0.5, time_decay=0.03)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)
    
    # Batches
    train_batches = []
    for i in range(0, len(X_train), config.BATCH_SIZE):
        end = min(i + config.BATCH_SIZE, len(X_train))
        bx = torch.tensor(np.stack(X_train[i:end]).astype(np.float32))
        by, bm = prepare_targets([y_train[j] for j in range(i, end)], horizon)
        train_batches.append((bx, by, bm))
    
    val_batches = []
    for i in range(0, len(X_val), config.BATCH_SIZE):
        end = min(i + config.BATCH_SIZE, len(X_val))
        bx = torch.tensor(np.stack(X_val[i:end]).astype(np.float32))
        by, bm = prepare_targets([y_val[j] for j in range(i, end)], horizon)
        val_batches.append((bx, by, bm))
    
    best_loss, best_state, bad = float('inf'), None, 0
    
    for epoch in tqdm(range(1, config.EPOCHS + 1)):
        model.train()
        train_losses = []
        for bx, by, bm in train_batches:
            bx, by, bm = bx.to(device), by.to(device), bm.to(device)
            pred = model(bx)
            loss = criterion(pred, by, bm)
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_losses.append(loss.item())
        
        model.eval()
        val_losses = []
        with torch.no_grad():
            for bx, by, bm in val_batches:
                bx, by, bm = bx.to(device), by.to(device), bm.to(device)
                pred = model(bx)
                val_losses.append(criterion(pred, by, bm).item())
        
        train_loss, val_loss = np.mean(train_losses), np.mean(val_losses)
        scheduler.step(val_loss)
        
        if epoch % 10 == 0:
            print(f"  Epoch {epoch}: train={train_loss:.4f}, val={val_loss:.4f}")
        
        if val_loss < best_loss:
            best_loss = val_loss
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            bad = 0
        else:
            bad += 1
            if bad >= config.PATIENCE:
                print(f"  Early stop at epoch {epoch}")
                break
    
    if best_state:
        model.load_state_dict(best_state)
    
    return model, best_loss

In [14]:
# ============================================================================
# MAIN PIPELINE
# ============================================================================

config = Config()

print("="*80)
print("STEP 2: BETTER FEATURES PIPELINE")
print("="*80)
print("\nAdding 30-40 advanced features to improve from 0.62 → 0.59-0.60")

# Load
print("\n[1/4] Loading data...")
train_input_files = [config.DATA_DIR / f"train/input_2023_w{w:02d}.csv" for w in range(1, 19)]
train_output_files = [config.DATA_DIR / f"train/output_2023_w{w:02d}.csv" for w in range(1, 19)]
train_input = pd.concat([pd.read_csv(f) for f in train_input_files if f.exists()])
train_output = pd.concat([pd.read_csv(f) for f in train_output_files if f.exists()])
test_input = pd.read_csv(config.DATA_DIR / "test_input.csv")
test_template = pd.read_csv(config.DATA_DIR / "test.csv")

STEP 2: BETTER FEATURES PIPELINE

Adding 30-40 advanced features to improve from 0.62 → 0.59-0.60

[1/4] Loading data...


In [ ]:
# Prepare with advanced features
print("\n[2/4] Preparing with ADVANCED features...")
sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids = prepare_sequences_with_advanced_features(
    train_input, train_output, is_training=True, window_size=config.WINDOW_SIZE
)
 
sequences = np.array(sequences, dtype=object)
targets_dx = np.array(targets_dx, dtype=object)
targets_dy = np.array(targets_dy, dtype=object)




[2/4] Preparing with ADVANCED features...

PREPARING SEQUENCES WITH ADVANCED FEATURES
Window size: 20
Step 1/3: Adding basic features...
Step 2/3: Adding advanced features...
Adding advanced features...
Total features after enhancement: 107
Step 3/3: Creating sequences...
Using 92 features (was ~50, now ~90)


Creating sequences: 100%|██████████| 46045/46045 [05:25<00:00, 141.40it/s]


Created 40272 sequences with 92 features each

[3/4] Training with enhanced features...

Fold 1/5
Training X-axis model...


TypeError: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'

In [13]:
# Train
print("\n[3/4] Training with enhanced features...")
groups = np.array([d['game_id'] for d in sequence_ids])
gkf = GroupKFold(n_splits=config.N_FOLDS)

models_x, models_y, scalers = [], [], []

for fold, (tr, va) in enumerate(gkf.split(sequences, groups=groups), 1):
    print(f"\n{'='*60}")
    print(f"Fold {fold}/{config.N_FOLDS}")
    print(f"{'='*60}")
    
    X_tr, X_va = sequences[tr], sequences[va]
    
    scaler = StandardScaler()
    scaler.fit(np.vstack([s for s in X_tr]))
    
    X_tr_sc = np.stack([scaler.transform(s) for s in X_tr])
    X_va_sc = np.stack([scaler.transform(s) for s in X_va])
    
    # Train X
    print("Training X-axis model...")
    mx, loss_x = train_model(
        X_tr_sc, targets_dx[tr], X_va_sc, targets_dx[va],
        X_tr[0].shape[-1], config.MAX_FUTURE_HORIZON, config
    )
    
    # Train Y
    print("Training Y-axis model...")
    my, loss_y = train_model(
        X_tr_sc, targets_dy[tr], X_va_sc, targets_dy[va],
        X_tr[0].shape[-1], config.MAX_FUTURE_HORIZON, config
    )
    
    models_x.append(mx)
    models_y.append(my)
    scalers.append(scaler)
    
    print(f"\nFold {fold} - X loss: {loss_x:.5f}, Y loss: {loss_y:.5f}")




[3/4] Training with enhanced features...

Fold 1/5
Training X-axis model...


  1%|          | 2/200 [00:22<37:54, 11.49s/it]


KeyboardInterrupt: 

In [ ]:
# Test predictions
print("\n[4/4] Creating test predictions...")
test_sequences, test_ids = prepare_sequences_with_advanced_features(
    test_input, test_template=test_template, is_training=False, window_size=config.WINDOW_SIZE
)

X_test = np.array(test_sequences, dtype=object)
x_last = np.array([s[-1, 0] for s in X_test])
y_last = np.array([s[-1, 1] for s in X_test])

# Ensemble predictions across folds
all_dx, all_dy = [], []
for mx, my, sc in zip(models_x, models_y, scalers):
    X_sc = np.stack([sc.transform(s) for s in X_test])
    X_t = torch.tensor(X_sc.astype(np.float32)).to(config.DEVICE)
    
    mx.eval()
    my.eval()
    
    with torch.no_grad():
        all_dx.append(mx(X_t).cpu().numpy())
        all_dy.append(my(X_t).cpu().numpy())

ens_dx = np.mean(all_dx, axis=0)
ens_dy = np.mean(all_dy, axis=0)

# Create submission
rows = []
H = ens_dx.shape[1]

for i, sid in enumerate(test_ids):
    fids = test_template[
        (test_template['game_id'] == sid['game_id']) &
        (test_template['play_id'] == sid['play_id']) &
        (test_template['nfl_id'] == sid['nfl_id'])
    ]['frame_id'].sort_values().tolist()
    
    for t, fid in enumerate(fids):
        tt = min(t, H - 1)
        px = np.clip(x_last[i] + ens_dx[i, tt], 0, 120)
        py = np.clip(y_last[i] + ens_dy[i, tt], 0, 53.3)
        
        rows.append({
            'id': f"{sid['game_id']}_{sid['play_id']}_{sid['nfl_id']}_{fid}",
            'x': px,
            'y': py
        })

submission = pd.DataFrame(rows)
submission.to_csv("submission.csv", index=False)

print("\n" + "="*80)
print("STEP 2 COMPLETE!")
print("="*80)
print(f"✓ Saved submission.csv")
print(f"  Rows: {len(submission)}")
print(f"  Features used: ~90 (was ~50)")
print(f"\nExpected improvement:")
print(f"  Before: 0.62 RMSE (baseline features)")
print(f"  After:  0.59-0.60 RMSE (with advanced features)")
print(f"\nNew feature groups added:")
print(f"  1. Distance rate features (3)")
print(f"  2. Target alignment features (3)")
print(f"  3. Multi-window rolling features (24)")
print(f"  4. Extended lag features (8)")
print(f"  5. Velocity change features (4)")
print(f"  6. Field position features (4)")
print(f"  7. Role-specific features (3)")
print(f"  8. Time-based features (2)")
print(f"\nTotal new features: ~40")



In [ ]:
input_df = train_input, 
output_df = train_output
is_training=True
window_size=config.WINDOW_SIZE

"""
Prepare sequences with ALL advanced features
"""
print(f"\n{'='*80}")
print(f"PREPARING SEQUENCES WITH ADVANCED FEATURES")
print(f"{'='*80}")
print(f"Window size: {window_size}")

input_df = input_df.copy()

# ==========================================
# BASIC FEATURES
# ==========================================
print("Step 1/3: Adding basic features...")

input_df['player_height_feet'] = input_df['player_height'].apply(height_to_feet)

dir_rad = np.deg2rad(input_df['dir'].fillna(0))
delta_t = 0.1
input_df['velocity_x'] = (input_df['s'] + 0.5 * input_df['a'] * delta_t) * np.sin(dir_rad)
input_df['velocity_y'] = (input_df['s'] + 0.5 * input_df['a'] * delta_t) * np.cos(dir_rad)
input_df['acceleration_x'] = input_df['a'] * np.sin(dir_rad)
input_df['acceleration_y'] = input_df['a'] * np.cos(dir_rad)

# Roles
input_df['is_offense'] = (input_df['player_side'] == 'Offense').astype(int)
input_df['is_defense'] = (input_df['player_side'] == 'Defense').astype(int)
input_df['is_receiver'] = (input_df['player_role'] == 'Targeted Receiver').astype(int)
input_df['is_coverage'] = (input_df['player_role'] == 'Defensive Coverage').astype(int)
input_df['is_passer'] = (input_df['player_role'] == 'Passer').astype(int)

# Physics
mass_kg = input_df['player_weight'].fillna(200.0) / 2.20462
input_df['momentum_x'] = input_df['velocity_x'] * mass_kg
input_df['momentum_y'] = input_df['velocity_y'] * mass_kg
input_df['kinetic_energy'] = 0.5 * mass_kg * (input_df['s'] ** 2)

# Ball features
if 'ball_land_x' in input_df.columns:
    ball_dx = input_df['ball_land_x'] - input_df['x']
    ball_dy = input_df['ball_land_y'] - input_df['y']
    input_df['distance_to_ball'] = np.sqrt(ball_dx**2 + ball_dy**2)
    input_df['angle_to_ball'] = np.arctan2(ball_dy, ball_dx)
    input_df['ball_direction_x'] = ball_dx / (input_df['distance_to_ball'] + 1e-6)
    input_df['ball_direction_y'] = ball_dy / (input_df['distance_to_ball'] + 1e-6)
    input_df['closing_speed'] = (
        input_df['velocity_x'] * input_df['ball_direction_x'] +
        input_df['velocity_y'] * input_df['ball_direction_y']
    )

# Sort for temporal
input_df = input_df.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
gcols = ['game_id', 'play_id', 'nfl_id']

# Original lag features (1-3)
for lag in [1, 2, 3]:
    input_df[f'x_lag{lag}'] = input_df.groupby(gcols)['x'].shift(lag)
    input_df[f'y_lag{lag}'] = input_df.groupby(gcols)['y'].shift(lag)
    input_df[f'velocity_x_lag{lag}'] = input_df.groupby(gcols)['velocity_x'].shift(lag)
    input_df[f'velocity_y_lag{lag}'] = input_df.groupby(gcols)['velocity_y'].shift(lag)

# EMA features
input_df['velocity_x_ema'] = input_df.groupby(gcols)['velocity_x'].transform(
    lambda x: x.ewm(alpha=0.3, adjust=False).mean()
)
input_df['velocity_y_ema'] = input_df.groupby(gcols)['velocity_y'].transform(
    lambda x: x.ewm(alpha=0.3, adjust=False).mean()
)
input_df['speed_ema'] = input_df.groupby(gcols)['s'].transform(
    lambda x: x.ewm(alpha=0.3, adjust=False).mean()
)

# ==========================================
# ADVANCED FEATURES (NEW!)
# ==========================================
print("Step 2/3: Adding advanced features...")
input_df = add_advanced_features(input_df)

# ==========================================
# FEATURE LIST (ENHANCED)
# ==========================================
print("Step 3/3: Creating sequences...")

feature_cols = [
    # Core (9)
    'x', 'y', 's', 'a', 'o', 'dir', 'frame_id', 'ball_land_x', 'ball_land_y',
    
    # Player (2)
    'player_height_feet', 'player_weight',
    
    # Motion (6)
    'velocity_x', 'velocity_y', 'acceleration_x', 'acceleration_y',
    'momentum_x', 'momentum_y', 'kinetic_energy',
    
    # Roles (5)
    'is_offense', 'is_defense', 'is_receiver', 'is_coverage', 'is_passer',
    
    # Ball (5)
    'distance_to_ball', 'angle_to_ball', 'ball_direction_x', 'ball_direction_y', 'closing_speed',
    
    # Original temporal (15)
    'x_lag1', 'y_lag1', 'velocity_x_lag1', 'velocity_y_lag1',
    'x_lag2', 'y_lag2', 'velocity_x_lag2', 'velocity_y_lag2',
    'x_lag3', 'y_lag3', 'velocity_x_lag3', 'velocity_y_lag3',
    'velocity_x_ema', 'velocity_y_ema', 'speed_ema',
    
    # NEW: Distance rate (3)
    'distance_to_ball_change', 'distance_to_ball_accel', 'time_to_intercept',
    
    # NEW: Target alignment (3)
    'velocity_alignment', 'velocity_perpendicular', 'accel_alignment',
    
    # NEW: Multi-window rolling (24)
    'velocity_x_roll3', 'velocity_x_std3', 'velocity_y_roll3', 'velocity_y_std3',
    's_roll3', 's_std3', 'a_roll3', 'a_std3',
    'velocity_x_roll5', 'velocity_x_std5', 'velocity_y_roll5', 'velocity_y_std5',
    's_roll5', 's_std5', 'a_roll5', 'a_std5',
    'velocity_x_roll10', 'velocity_x_std10', 'velocity_y_roll10', 'velocity_y_std10',
    's_roll10', 's_std10', 'a_roll10', 'a_std10',
    
    # NEW: Extended lags (8)
    'x_lag4', 'y_lag4', 'velocity_x_lag4', 'velocity_y_lag4',
    'x_lag5', 'y_lag5', 'velocity_x_lag5', 'velocity_y_lag5',
    
    # NEW: Velocity changes (4)
    'velocity_x_change', 'velocity_y_change', 'speed_change', 'direction_change',
    
    # NEW: Field position (4)
    'dist_from_sideline', 'dist_from_endzone',
    
    # NEW: Role-specific (3)
    'receiver_optimality', 'receiver_deviation', 'defender_closing_speed',
    
    # NEW: Time (2)
    'frames_elapsed', 'normalized_time',
]

# Filter to existing
feature_cols = [c for c in feature_cols if c in input_df.columns]
print(f"Using {len(feature_cols)} features (was ~50, now ~90)")

# ==========================================
# CREATE SEQUENCES
# ==========================================
input_df.set_index(['game_id', 'play_id', 'nfl_id'], inplace=True)
grouped = input_df.groupby(level=['game_id', 'play_id', 'nfl_id'])

target_rows = output_df if is_training else test_template
target_groups = target_rows[['game_id', 'play_id', 'nfl_id']].drop_duplicates()

sequences, targets_dx, targets_dy, targets_frame_ids, sequence_ids = [], [], [], [], []

for _, row in tqdm(target_groups.iterrows(), total=len(target_groups), desc="Creating sequences"):
    key = (row['game_id'], row['play_id'], row['nfl_id'])
    
    try:
        group_df = grouped.get_group(key)
    except KeyError:
        continue
    
    input_window = group_df.tail(window_size)
    
    if len(input_window) < window_size:
        if is_training:
            continue
        pad_len = window_size - len(input_window)
        pad_df = pd.DataFrame(np.nan, index=range(pad_len), columns=input_window.columns)
        input_window = pd.concat([pad_df, input_window], ignore_index=True)
    
    input_window = input_window.fillna(group_df.mean(numeric_only=True))
    seq = input_window[feature_cols].values
    
    if np.isnan(seq).any():
        if is_training:
            continue
        seq = np.nan_to_num(seq, nan=0.0)
    
    sequences.append(seq)
    
    if is_training:
        out_grp = output_df[
            (output_df['game_id']==row['game_id']) &
            (output_df['play_id']==row['play_id']) &
            (output_df['nfl_id']==row['nfl_id'])
        ].sort_values('frame_id')
        
        last_x = input_window.iloc[-1]['x']
        last_y = input_window.iloc[-1]['y']
        
        dx = out_grp['x'].values - last_x
        dy = out_grp['y'].values - last_y
        
        targets_dx.append(dx)
        targets_dy.append(dy)
        targets_frame_ids.append(out_grp['frame_id'].values)
    
    sequence_ids.append({
        'game_id': key[0],
        'play_id': key[1],
        'nfl_id': key[2],
        'frame_id': input_window.iloc[-1]['frame_id']
    })

print(f"Created {len(sequences)} sequences with {len(feature_cols)} features each")
